In [ ]:
import numpy as np
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import kendalltau
import statistics 
import pandas as pd
import math
import os
import random
import torch
import torch.nn as nn
import sys
from statistics import mean, median 
from sklearn import svm
import torch.nn.functional as F
import torch
from geomloss import SamplesLoss
from scipy.spatial import distance
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics

In [ ]:

def mmd_rbf(X, Y, gamma=1.0):
    """MMD using rbf (gaussian) kernel (i.e., k(x,y) = exp(-gamma * ||x-y||^2 / 2))

    Arguments:
        X {[n_sample1, dim]} -- [X matrix]
        Y {[n_sample2, dim]} -- [Y matrix]

    Keyword Arguments:
        gamma {float} -- [kernel parameter] (default: {1.0})

    Returns:
        [scalar] -- [MMD value]
    """
    #source = F.normalize(source, p=2) #
    #target = F.normalize(target, p=2)
    #X = X.cpu()
    #Y = Y.cpu()    
    XX = metrics.pairwise.rbf_kernel(X, X, gamma)
    YY = metrics.pairwise.rbf_kernel(Y, Y, gamma)
    XY = metrics.pairwise.rbf_kernel(X, Y, gamma)
    return XX.mean() + YY.mean() - 2 * XY.mean()





def CORAL(source, target):
    #source = F.normalize(source, p=2)
    #target = F.normalize(target, p=2)
    source = torch.tensor(source)
    target = torch.tensor(target)
    d = source.data.shape[1]
    ns, nt = source.data.shape[0], target.data.shape[0]
    # source covariance
    xm = torch.mean(source, 0, keepdim=True) - source
    xc = xm.t() @ xm / (ns - 1)

    # target covariance
    xmt = torch.mean(target, 0, keepdim=True) - target
    xct = xmt.t() @ xmt / (nt - 1)

    # frobenius norm between source and target
    loss = torch.mul((xc - xct), (xc - xct))
    loss = torch.sum(loss) / (4*d*d)
    return float(loss)


def CORAL_norm(source, target):
    source = torch.tensor(source)
    target = torch.tensor(target)
    
    source = F.normalize(source, p=2)
    target = F.normalize(target, p=2)
    
    
    d = source.data.shape[1]
    ns, nt = source.data.shape[0], target.data.shape[0]
    # source covariance
    xm = torch.mean(source, 0, keepdim=True) - source
    xc = xm.t() @ xm / (ns - 1)

    # target covariance
    xmt = torch.mean(target, 0, keepdim=True) - target
    xct = xmt.t() @ xmt / (nt - 1)

    # frobenius norm between source and target
    loss = torch.mul((xc - xct), (xc - xct))
    loss = torch.sum(loss) / (4*d*d)
    return float(loss)

def L2_loss(source, target): 
    #source = F.normalize(source, p=2)
    #target = F.normalize(target, p=2)
    source = torch.tensor(source)
    target = torch.tensor(target)    
    loss = torch.cdist(source, target, p=2.0).mean()
    return float(loss)

def L2_loss_norm(source, target): 
    source = torch.tensor(source)
    target = torch.tensor(target)    
    source = F.normalize(source, p=2)
    target = F.normalize(target, p=2)
    loss = torch.cdist(source, target, p=2.0)
    loss = float(loss.mean())
    return loss
    
def L2_squared_loss(source, target): 
    source = torch.tensor(source)
    target = torch.tensor(target)    
    #source = F.normalize(source, p=2)
    #target = F.normalize(target, p=2)
    loss = torch.cdist(source, target, p=2.0)**2
    loss = float(loss.mean())
    return loss 
    
def L2_squared_loss_norm(source, target):
    source = torch.tensor(source)
    target = torch.tensor(target)     
    source = F.normalize(source, p=2)
    target = F.normalize(target, p=2)
    loss = torch.cdist(source, target, p=2.0)**2
    loss = float(loss.mean())
    return loss 
    
def Dot_loss(source, target): 
    source = torch.tensor(source)
    target = torch.tensor(target)    
    dot_sim = torch.matmul(source,target.T)
    dot_loss = -float(dot_sim.mean())
    return dot_loss

def Dot_loss_norm(source, target): 
    source = torch.tensor(source)
    target = torch.tensor(target)    
    source = F.normalize(source, p=2)
    target = F.normalize(target, p=2)
    dot_sim = torch.matmul(source,target.T)
    dot_loss = -float(dot_sim.mean())
    return dot_loss



def Cos_loss(source, target): 
    source = torch.tensor(source)
    target = torch.tensor(target)    

    cos_sim = torch.nn.functional.cosine_similarity(source.unsqueeze(1), target.unsqueeze(0), dim=2)
    #print(cos_sim)

    distance = 1 - cos_sim
    loss = float(distance.mean())
    return loss


In [ ]:
codebert4vdcwe_small = 'codebert4vdcwe-small/'
codebert4vdcwe_base = 'codebert4vdcwe-base/'
codegen4vdcwe_small = 'codegen4vdcwe-small/'
codegen4vdcwe_base = 'codegen4vdcwe-base/'
codet54vdcwe_small = 'codet54vdcwe-small/'
codet54vdcwe_base = 'codet54vdcwe-base/'
codet54vdcwe_large = 'codet54vdcwe-large/'
polycoder4vdcwe_small = 'polycoder4vdcwe-small/'
polycoder4vdcwe_base = 'polycoder4vdcwe-base/'
polycoder4vdcwe_large = 'polycoder4vdcwe-large/'



trained_codexglue_path = 'CLS/trained_codexglue/CLS/'
trained_reveal_path = 'CLS/trained_reveal/CLS/'
trained_cwe119_path = 'CLS/trained_cwe119/CLS/'
trained_cwe399_path = 'CLS/trained_cwe399/CLS/'


In [ ]:

codebert4vdcwe_small_codexglue_train = {}
codebert4vdcwe_small_codexglue_test = {}
codebert4vdcwe_base_codexglue_train = {}
codebert4vdcwe_base_codexglue_test = {}
codegen4vdcwe_small_codexglue_train = {}
codegen4vdcwe_small_codexglue_test = {}
codegen4vdcwe_base_codexglue_train = {}
codegen4vdcwe_base_codexglue_test = {}
codet54vdcwe_small_codexglue_train = {}
codet54vdcwe_small_codexglue_test = {}
codet54vdcwe_base_codexglue_train = {}
codet54vdcwe_base_codexglue_test = {}
codet54vdcwe_large_codexglue_train = {}
codet54vdcwe_large_codexglue_test = {}
polycoder4vdcwe_small_codexglue_train = {}
polycoder4vdcwe_small_codexglue_test = {}
polycoder4vdcwe_base_codexglue_train = {}
polycoder4vdcwe_base_codexglue_test = {}
polycoder4vdcwe_large_codexglue_train = {}
polycoder4vdcwe_large_codexglue_test = {}

codebert4vdcwe_small_codexglue_distance = {}
codebert4vdcwe_base_codexglue_distance = {}
codegen4vdcwe_small_codexglue_distance = {}
codegen4vdcwe_base_codexglue_distance = {}
codet54vdcwe_small_codexglue_distance = {}
codet54vdcwe_base_codexglue_distance = {}
codet54vdcwe_large_codexglue_distance = {}
polycoder4vdcwe_small_codexglue_distance = {}
polycoder4vdcwe_base_codexglue_distance = {}
polycoder4vdcwe_large_codexglue_distance = {}

for i in range(200):  
    codebert4vdcwe_small_codexglue_train[i] = np.load(codebert4vdcwe_small+trained_codexglue_path+'codexglue_train{}'.format(i)+'-data.npy')
    codebert4vdcwe_small_codexglue_test[i] = np.load(codebert4vdcwe_small+trained_codexglue_path+'codexglue_test{}'.format(i)+'-data.npy')
    #print(np.shape(codebert4vdcwe_small_codexglue_train[i]))  
    #print(np.shape(codebert4vdcwe_small_codexglue_test[i]))
    codebert4vdcwe_small_codexglue_distance[i] = [mmd_rbf(codebert4vdcwe_small_codexglue_train[i],codebert4vdcwe_small_codexglue_test[i]),
                                                  L2_loss(codebert4vdcwe_small_codexglue_train[i],codebert4vdcwe_small_codexglue_test[i]),
                                                  L2_loss_norm(codebert4vdcwe_small_codexglue_train[i],codebert4vdcwe_small_codexglue_test[i]),
                                                  L2_squared_loss(codebert4vdcwe_small_codexglue_train[i],codebert4vdcwe_small_codexglue_test[i]),
                                                  L2_squared_loss_norm(codebert4vdcwe_small_codexglue_train[i],codebert4vdcwe_small_codexglue_test[i]),
                                                  Dot_loss(codebert4vdcwe_small_codexglue_train[i],codebert4vdcwe_small_codexglue_test[i]),
                                                  Dot_loss_norm(codebert4vdcwe_small_codexglue_train[i],codebert4vdcwe_small_codexglue_test[i])]
    
    codebert4vdcwe_base_codexglue_train[i] = np.load(codebert4vdcwe_base+trained_codexglue_path+'codexglue_train{}'.format(i)+'-data.npy')
    codebert4vdcwe_base_codexglue_test[i] = np.load(codebert4vdcwe_base+trained_codexglue_path+'codexglue_test{}'.format(i)+'-data.npy')
    #print(np.shape(codebert4vdcwe_base_codexglue_train[i]))  
    #print(np.shape(codebert4vdcwe_base_codexglue_test[i]))
    codebert4vdcwe_base_codexglue_distance[i] = [mmd_rbf(codebert4vdcwe_base_codexglue_train[i],codebert4vdcwe_base_codexglue_test[i]),
                                                  L2_loss(codebert4vdcwe_base_codexglue_train[i],codebert4vdcwe_base_codexglue_test[i]),
                                                  L2_loss_norm(codebert4vdcwe_base_codexglue_train[i],codebert4vdcwe_base_codexglue_test[i]),
                                                  L2_squared_loss(codebert4vdcwe_base_codexglue_train[i],codebert4vdcwe_base_codexglue_test[i]),
                                                  L2_squared_loss_norm(codebert4vdcwe_base_codexglue_train[i],codebert4vdcwe_base_codexglue_test[i]),
                                                  Dot_loss(codebert4vdcwe_base_codexglue_train[i],codebert4vdcwe_base_codexglue_test[i]),
                                                  Dot_loss_norm(codebert4vdcwe_base_codexglue_train[i],codebert4vdcwe_base_codexglue_test[i])]
              
    codegen4vdcwe_small_codexglue_train[i] = np.load(codegen4vdcwe_small+trained_codexglue_path+'codexglue_train{}'.format(i)+'-data.npy')
    codegen4vdcwe_small_codexglue_test[i] = np.load(codegen4vdcwe_small+trained_codexglue_path+'codexglue_test{}'.format(i)+'-data.npy')
    #print(np.shape(codegen4vdcwe_small_codexglue_train[i]))  
    #print(np.shape(codegen4vdcwe_small_codexglue_test[i]))
    codegen4vdcwe_small_codexglue_distance[i] = [mmd_rbf(codegen4vdcwe_small_codexglue_train[i],codegen4vdcwe_small_codexglue_test[i]),
                                                  L2_loss(codegen4vdcwe_small_codexglue_train[i],codegen4vdcwe_small_codexglue_test[i]),
                                                  L2_loss_norm(codegen4vdcwe_small_codexglue_train[i],codegen4vdcwe_small_codexglue_test[i]),
                                                  L2_squared_loss(codegen4vdcwe_small_codexglue_train[i],codegen4vdcwe_small_codexglue_test[i]),
                                                  L2_squared_loss_norm(codegen4vdcwe_small_codexglue_train[i],codegen4vdcwe_small_codexglue_test[i]),
                                                  Dot_loss(codegen4vdcwe_small_codexglue_train[i],codegen4vdcwe_small_codexglue_test[i]),
                                                  Dot_loss_norm(codegen4vdcwe_small_codexglue_train[i],codegen4vdcwe_small_codexglue_test[i])]
                            
                           
    
    
    codegen4vdcwe_base_codexglue_train[i] = np.load(codegen4vdcwe_base+trained_codexglue_path+'codexglue_train{}'.format(i)+'-data.npy')
    codegen4vdcwe_base_codexglue_test[i] = np.load(codegen4vdcwe_base+trained_codexglue_path+'codexglue_test{}'.format(i)+'-data.npy')
    #print(np.shape(codegen4vdcwe_base_codexglue_train[i]))  
    #print(np.shape(codegen4vdcwe_base_codexglue_test[i]))
    codegen4vdcwe_base_codexglue_distance[i] = [mmd_rbf(codegen4vdcwe_base_codexglue_train[i],codegen4vdcwe_base_codexglue_test[i]),
                                                  L2_loss(codegen4vdcwe_base_codexglue_train[i],codegen4vdcwe_base_codexglue_test[i]),
                                                  L2_loss_norm(codegen4vdcwe_base_codexglue_train[i],codegen4vdcwe_base_codexglue_test[i]),
                                                  L2_squared_loss(codegen4vdcwe_base_codexglue_train[i],codegen4vdcwe_base_codexglue_test[i]),
                                                  L2_squared_loss_norm(codegen4vdcwe_base_codexglue_train[i],codegen4vdcwe_base_codexglue_test[i]),
                                                  Dot_loss(codegen4vdcwe_base_codexglue_train[i],codegen4vdcwe_base_codexglue_test[i]),
                                                  Dot_loss_norm(codegen4vdcwe_base_codexglue_train[i],codegen4vdcwe_base_codexglue_test[i])]
        
    codet54vdcwe_small_codexglue_train[i] = np.load(codet54vdcwe_small+trained_codexglue_path+'codexglue_train{}'.format(i)+'-data.npy')
    codet54vdcwe_small_codexglue_test[i] = np.load(codet54vdcwe_small+trained_codexglue_path+'codexglue_test{}'.format(i)+'-data.npy')
    #print(np.shape(codet54vdcwe_small_codexglue_train[i]))  
    #print(np.shape(codet54vdcwe_small_codexglue_test[i]))
    codet54vdcwe_small_codexglue_distance[i] = [mmd_rbf(codet54vdcwe_small_codexglue_train[i],codet54vdcwe_small_codexglue_test[i]),
                                                  L2_loss(codet54vdcwe_small_codexglue_train[i],codet54vdcwe_small_codexglue_test[i]),
                                                  L2_loss_norm(codet54vdcwe_small_codexglue_train[i],codet54vdcwe_small_codexglue_test[i]),
                                                  L2_squared_loss(codet54vdcwe_small_codexglue_train[i],codet54vdcwe_small_codexglue_test[i]),
                                                  L2_squared_loss_norm(codet54vdcwe_small_codexglue_train[i],codet54vdcwe_small_codexglue_test[i]),
                                                  Dot_loss(codet54vdcwe_small_codexglue_train[i],codet54vdcwe_small_codexglue_test[i]),
                                                  Dot_loss_norm(codet54vdcwe_small_codexglue_train[i],codet54vdcwe_small_codexglue_test[i])]

    codet54vdcwe_base_codexglue_train[i] = np.load(codet54vdcwe_base+trained_codexglue_path+'codexglue_train{}'.format(i)+'-data.npy')
    codet54vdcwe_base_codexglue_test[i] = np.load(codet54vdcwe_base+trained_codexglue_path+'codexglue_test{}'.format(i)+'-data.npy')
    #print(np.shape(codet54vdcwe_base_codexglue_train[i]))  
    #print(np.shape(codet54vdcwe_base_codexglue_test[i]))
    codet54vdcwe_base_codexglue_distance[i] = [mmd_rbf(codet54vdcwe_base_codexglue_train[i],codet54vdcwe_base_codexglue_test[i]),
                                                  L2_loss(codet54vdcwe_base_codexglue_train[i],codet54vdcwe_base_codexglue_test[i]),
                                                  L2_loss_norm(codet54vdcwe_base_codexglue_train[i],codet54vdcwe_base_codexglue_test[i]),
                                                  L2_squared_loss(codet54vdcwe_base_codexglue_train[i],codet54vdcwe_base_codexglue_test[i]),
                                                  L2_squared_loss_norm(codet54vdcwe_base_codexglue_train[i],codet54vdcwe_base_codexglue_test[i]),
                                                  Dot_loss(codet54vdcwe_base_codexglue_train[i],codet54vdcwe_base_codexglue_test[i]),
                                                  Dot_loss_norm(codet54vdcwe_base_codexglue_train[i],codet54vdcwe_base_codexglue_test[i])]
    
    codet54vdcwe_large_codexglue_train[i] = np.load(codet54vdcwe_large+trained_codexglue_path+'codexglue_train{}'.format(i)+'-data.npy')
    codet54vdcwe_large_codexglue_test[i] = np.load(codet54vdcwe_large+trained_codexglue_path+'codexglue_test{}'.format(i)+'-data.npy')
    #print(np.shape(codet54vdcwe_large_codexglue_train[i]))  
    #print(np.shape(codet54vdcwe_large_codexglue_test[i]))
    codet54vdcwe_large_codexglue_distance[i] = [mmd_rbf(codet54vdcwe_large_codexglue_train[i],codet54vdcwe_large_codexglue_test[i]),
                                                  L2_loss(codet54vdcwe_large_codexglue_train[i],codet54vdcwe_large_codexglue_test[i]),
                                                  L2_loss_norm(codet54vdcwe_large_codexglue_train[i],codet54vdcwe_large_codexglue_test[i]),
                                                  L2_squared_loss(codet54vdcwe_large_codexglue_train[i],codet54vdcwe_large_codexglue_test[i]),
                                                  L2_squared_loss_norm(codet54vdcwe_large_codexglue_train[i],codet54vdcwe_large_codexglue_test[i]),
                                                  Dot_loss(codet54vdcwe_large_codexglue_train[i],codet54vdcwe_large_codexglue_test[i]),
                                                  Dot_loss_norm(codet54vdcwe_large_codexglue_train[i],codet54vdcwe_large_codexglue_test[i])]

    polycoder4vdcwe_small_codexglue_train[i] = np.load(polycoder4vdcwe_small+trained_codexglue_path+'codexglue_train{}'.format(i)+'-data.npy')
    polycoder4vdcwe_small_codexglue_test[i] = np.load(polycoder4vdcwe_small+trained_codexglue_path+'codexglue_test{}'.format(i)+'-data.npy')
    #print(np.shape(polycoder4vdcwe_small_codexglue_train[i]))  
    #print(np.shape(polycoder4vdcwe_small_codexglue_test[i]))
    polycoder4vdcwe_small_codexglue_distance[i] = [mmd_rbf(polycoder4vdcwe_small_codexglue_train[i],polycoder4vdcwe_small_codexglue_test[i]),
                                                  L2_loss(polycoder4vdcwe_small_codexglue_train[i],polycoder4vdcwe_small_codexglue_test[i]),
                                                  L2_loss_norm(polycoder4vdcwe_small_codexglue_train[i],polycoder4vdcwe_small_codexglue_test[i]),
                                                  L2_squared_loss(polycoder4vdcwe_small_codexglue_train[i],polycoder4vdcwe_small_codexglue_test[i]),
                                                  L2_squared_loss_norm(polycoder4vdcwe_small_codexglue_train[i],polycoder4vdcwe_small_codexglue_test[i]),
                                                  Dot_loss(polycoder4vdcwe_small_codexglue_train[i],polycoder4vdcwe_small_codexglue_test[i]),
                                                  Dot_loss_norm(polycoder4vdcwe_small_codexglue_train[i],polycoder4vdcwe_small_codexglue_test[i])]
        
    
    polycoder4vdcwe_base_codexglue_train[i] = np.load(polycoder4vdcwe_base+trained_codexglue_path+'codexglue_train{}'.format(i)+'-data.npy')
    polycoder4vdcwe_base_codexglue_test[i] = np.load(polycoder4vdcwe_base+trained_codexglue_path+'codexglue_test{}'.format(i)+'-data.npy')
    #print(np.shape(polycoder4vdcwe_base_codexglue_train[i]))  
    #print(np.shape(polycoder4vdcwe_base_codexglue_test[i]))
    polycoder4vdcwe_base_codexglue_distance[i] = [mmd_rbf(polycoder4vdcwe_base_codexglue_train[i],polycoder4vdcwe_base_codexglue_test[i]),
                                                  L2_loss(polycoder4vdcwe_base_codexglue_train[i],polycoder4vdcwe_base_codexglue_test[i]),
                                                  L2_loss_norm(polycoder4vdcwe_base_codexglue_train[i],polycoder4vdcwe_base_codexglue_test[i]),
                                                  L2_squared_loss(polycoder4vdcwe_base_codexglue_train[i],polycoder4vdcwe_base_codexglue_test[i]),
                                                  L2_squared_loss_norm(polycoder4vdcwe_base_codexglue_train[i],polycoder4vdcwe_base_codexglue_test[i]),
                                                  Dot_loss(polycoder4vdcwe_base_codexglue_train[i],polycoder4vdcwe_base_codexglue_test[i]),
                                                  Dot_loss_norm(polycoder4vdcwe_base_codexglue_train[i],polycoder4vdcwe_base_codexglue_test[i])]
        
    
    
    polycoder4vdcwe_large_codexglue_train[i] = np.load(polycoder4vdcwe_large+trained_codexglue_path+'codexglue_train{}'.format(i)+'-data.npy')
    polycoder4vdcwe_large_codexglue_test[i] = np.load(polycoder4vdcwe_large+trained_codexglue_path+'codexglue_test{}'.format(i)+'-data.npy')
    #print(np.shape(polycoder4vdcwe_large_codexglue_train[i]))  
    #print(np.shape(polycoder4vdcwe_large_codexglue_test[i]))
    polycoder4vdcwe_large_codexglue_distance[i] = [mmd_rbf(polycoder4vdcwe_large_codexglue_train[i],polycoder4vdcwe_large_codexglue_test[i]),
                                                  L2_loss(polycoder4vdcwe_large_codexglue_train[i],polycoder4vdcwe_large_codexglue_test[i]),
                                                  L2_loss_norm(polycoder4vdcwe_large_codexglue_train[i],polycoder4vdcwe_large_codexglue_test[i]),
                                                  L2_squared_loss(polycoder4vdcwe_large_codexglue_train[i],polycoder4vdcwe_large_codexglue_test[i]),
                                                  L2_squared_loss_norm(polycoder4vdcwe_large_codexglue_train[i],polycoder4vdcwe_large_codexglue_test[i]),
                                                  Dot_loss(polycoder4vdcwe_large_codexglue_train[i],polycoder4vdcwe_large_codexglue_test[i]),
                                                  Dot_loss_norm(polycoder4vdcwe_large_codexglue_train[i],polycoder4vdcwe_large_codexglue_test[i])]

    codebert4vdcwe_small_codexglue_train[i] = None
    codebert4vdcwe_small_codexglue_test[i] = None
    codebert4vdcwe_base_codexglue_train[i] = None
    codebert4vdcwe_base_codexglue_test[i] = None
    codegen4vdcwe_small_codexglue_train[i] = None
    codegen4vdcwe_small_codexglue_test[i] = None
    codegen4vdcwe_base_codexglue_train[i] = None
    codegen4vdcwe_base_codexglue_test[i] = None
    codet54vdcwe_small_codexglue_train[i] = None
    codet54vdcwe_small_codexglue_test[i] = None
    codet54vdcwe_base_codexglue_train[i] = None
    codet54vdcwe_base_codexglue_test[i] = None
    codet54vdcwe_large_codexglue_train[i] = None
    codet54vdcwe_large_codexglue_test[i] = None
    polycoder4vdcwe_small_codexglue_train[i] = None
    polycoder4vdcwe_small_codexglue_test[i] = None
    polycoder4vdcwe_base_codexglue_train[i] = None
    polycoder4vdcwe_base_codexglue_test[i] = None
    polycoder4vdcwe_large_codexglue_train[i] = None
    polycoder4vdcwe_large_codexglue_test[i] = None       

In [ ]:

codebert4vdcwe_small_reveal_train = {}
codebert4vdcwe_small_reveal_test = {}
codebert4vdcwe_base_reveal_train = {}
codebert4vdcwe_base_reveal_test = {}
codegen4vdcwe_small_reveal_train = {}
codegen4vdcwe_small_reveal_test = {}
codegen4vdcwe_base_reveal_train = {}
codegen4vdcwe_base_reveal_test = {}
codet54vdcwe_small_reveal_train = {}
codet54vdcwe_small_reveal_test = {}
codet54vdcwe_base_reveal_train = {}
codet54vdcwe_base_reveal_test = {}
codet54vdcwe_large_reveal_train = {}
codet54vdcwe_large_reveal_test = {}
polycoder4vdcwe_small_reveal_train = {}
polycoder4vdcwe_small_reveal_test = {}
polycoder4vdcwe_base_reveal_train = {}
polycoder4vdcwe_base_reveal_test = {}
polycoder4vdcwe_large_reveal_train = {}
polycoder4vdcwe_large_reveal_test = {}

codebert4vdcwe_small_reveal_distance = {}
codebert4vdcwe_base_reveal_distance = {}
codegen4vdcwe_small_reveal_distance = {}
codegen4vdcwe_base_reveal_distance = {}
codet54vdcwe_small_reveal_distance = {}
codet54vdcwe_base_reveal_distance = {}
codet54vdcwe_large_reveal_distance = {}
polycoder4vdcwe_small_reveal_distance = {}
polycoder4vdcwe_base_reveal_distance = {}
polycoder4vdcwe_large_reveal_distance = {}

for i in range(200):  
    codebert4vdcwe_small_reveal_train[i] = np.load(codebert4vdcwe_small+trained_reveal_path+'reveal_train{}'.format(i)+'-data.npy')
    codebert4vdcwe_small_reveal_test[i] = np.load(codebert4vdcwe_small+trained_reveal_path+'reveal_test{}'.format(i)+'-data.npy')
    codebert4vdcwe_small_reveal_distance[i] = [mmd_rbf(codebert4vdcwe_small_reveal_train[i],codebert4vdcwe_small_reveal_test[i]),
                                                  L2_loss(codebert4vdcwe_small_reveal_train[i],codebert4vdcwe_small_reveal_test[i]),
                                                  L2_loss_norm(codebert4vdcwe_small_reveal_train[i],codebert4vdcwe_small_reveal_test[i]),
                                                  L2_squared_loss(codebert4vdcwe_small_reveal_train[i],codebert4vdcwe_small_reveal_test[i]),
                                                  L2_squared_loss_norm(codebert4vdcwe_small_reveal_train[i],codebert4vdcwe_small_reveal_test[i]),
                                                  Dot_loss(codebert4vdcwe_small_reveal_train[i],codebert4vdcwe_small_reveal_test[i]),
                                                  Dot_loss_norm(codebert4vdcwe_small_reveal_train[i],codebert4vdcwe_small_reveal_test[i])]
    
    codebert4vdcwe_base_reveal_train[i] = np.load(codebert4vdcwe_base+trained_reveal_path+'reveal_train{}'.format(i)+'-data.npy')
    codebert4vdcwe_base_reveal_test[i] = np.load(codebert4vdcwe_base+trained_reveal_path+'reveal_test{}'.format(i)+'-data.npy')
    codebert4vdcwe_base_reveal_distance[i] = [mmd_rbf(codebert4vdcwe_base_reveal_train[i],codebert4vdcwe_base_reveal_test[i]),
                                                  L2_loss(codebert4vdcwe_base_reveal_train[i],codebert4vdcwe_base_reveal_test[i]),
                                                  L2_loss_norm(codebert4vdcwe_base_reveal_train[i],codebert4vdcwe_base_reveal_test[i]),
                                                  L2_squared_loss(codebert4vdcwe_base_reveal_train[i],codebert4vdcwe_base_reveal_test[i]),
                                                  L2_squared_loss_norm(codebert4vdcwe_base_reveal_train[i],codebert4vdcwe_base_reveal_test[i]),
                                                  Dot_loss(codebert4vdcwe_base_reveal_train[i],codebert4vdcwe_base_reveal_test[i]),
                                                  Dot_loss_norm(codebert4vdcwe_base_reveal_train[i],codebert4vdcwe_base_reveal_test[i])]
              
    codegen4vdcwe_small_reveal_train[i] = np.load(codegen4vdcwe_small+trained_reveal_path+'reveal_train{}'.format(i)+'-data.npy')
    codegen4vdcwe_small_reveal_test[i] = np.load(codegen4vdcwe_small+trained_reveal_path+'reveal_test{}'.format(i)+'-data.npy')
    codegen4vdcwe_small_reveal_distance[i] = [mmd_rbf(codegen4vdcwe_small_reveal_train[i],codegen4vdcwe_small_reveal_test[i]),
                                                  L2_loss(codegen4vdcwe_small_reveal_train[i],codegen4vdcwe_small_reveal_test[i]),
                                                  L2_loss_norm(codegen4vdcwe_small_reveal_train[i],codegen4vdcwe_small_reveal_test[i]),
                                                  L2_squared_loss(codegen4vdcwe_small_reveal_train[i],codegen4vdcwe_small_reveal_test[i]),
                                                  L2_squared_loss_norm(codegen4vdcwe_small_reveal_train[i],codegen4vdcwe_small_reveal_test[i]),
                                                  Dot_loss(codegen4vdcwe_small_reveal_train[i],codegen4vdcwe_small_reveal_test[i]),
                                                  Dot_loss_norm(codegen4vdcwe_small_reveal_train[i],codegen4vdcwe_small_reveal_test[i])]
                            
                           
    
    
    codegen4vdcwe_base_reveal_train[i] = np.load(codegen4vdcwe_base+trained_reveal_path+'reveal_train{}'.format(i)+'-data.npy')
    codegen4vdcwe_base_reveal_test[i] = np.load(codegen4vdcwe_base+trained_reveal_path+'reveal_test{}'.format(i)+'-data.npy')
    codegen4vdcwe_base_reveal_distance[i] = [mmd_rbf(codegen4vdcwe_base_reveal_train[i],codegen4vdcwe_base_reveal_test[i]),
                                                  L2_loss(codegen4vdcwe_base_reveal_train[i],codegen4vdcwe_base_reveal_test[i]),
                                                  L2_loss_norm(codegen4vdcwe_base_reveal_train[i],codegen4vdcwe_base_reveal_test[i]),
                                                  L2_squared_loss(codegen4vdcwe_base_reveal_train[i],codegen4vdcwe_base_reveal_test[i]),
                                                  L2_squared_loss_norm(codegen4vdcwe_base_reveal_train[i],codegen4vdcwe_base_reveal_test[i]),
                                                  Dot_loss(codegen4vdcwe_base_reveal_train[i],codegen4vdcwe_base_reveal_test[i]),
                                                  Dot_loss_norm(codegen4vdcwe_base_reveal_train[i],codegen4vdcwe_base_reveal_test[i])]
        
    codet54vdcwe_small_reveal_train[i] = np.load(codet54vdcwe_small+trained_reveal_path+'reveal_train{}'.format(i)+'-data.npy')
    codet54vdcwe_small_reveal_test[i] = np.load(codet54vdcwe_small+trained_reveal_path+'reveal_test{}'.format(i)+'-data.npy')
    codet54vdcwe_small_reveal_distance[i] = [mmd_rbf(codet54vdcwe_small_reveal_train[i],codet54vdcwe_small_reveal_test[i]),
                                                  L2_loss(codet54vdcwe_small_reveal_train[i],codet54vdcwe_small_reveal_test[i]),
                                                  L2_loss_norm(codet54vdcwe_small_reveal_train[i],codet54vdcwe_small_reveal_test[i]),
                                                  L2_squared_loss(codet54vdcwe_small_reveal_train[i],codet54vdcwe_small_reveal_test[i]),
                                                  L2_squared_loss_norm(codet54vdcwe_small_reveal_train[i],codet54vdcwe_small_reveal_test[i]),
                                                  Dot_loss(codet54vdcwe_small_reveal_train[i],codet54vdcwe_small_reveal_test[i]),
                                                  Dot_loss_norm(codet54vdcwe_small_reveal_train[i],codet54vdcwe_small_reveal_test[i])]

    codet54vdcwe_base_reveal_train[i] = np.load(codet54vdcwe_base+trained_reveal_path+'reveal_train{}'.format(i)+'-data.npy')
    codet54vdcwe_base_reveal_test[i] = np.load(codet54vdcwe_base+trained_reveal_path+'reveal_test{}'.format(i)+'-data.npy')
    codet54vdcwe_base_reveal_distance[i] = [mmd_rbf(codet54vdcwe_base_reveal_train[i],codet54vdcwe_base_reveal_test[i]),
                                                  L2_loss(codet54vdcwe_base_reveal_train[i],codet54vdcwe_base_reveal_test[i]),
                                                  L2_loss_norm(codet54vdcwe_base_reveal_train[i],codet54vdcwe_base_reveal_test[i]),
                                                  L2_squared_loss(codet54vdcwe_base_reveal_train[i],codet54vdcwe_base_reveal_test[i]),
                                                  L2_squared_loss_norm(codet54vdcwe_base_reveal_train[i],codet54vdcwe_base_reveal_test[i]),
                                                  Dot_loss(codet54vdcwe_base_reveal_train[i],codet54vdcwe_base_reveal_test[i]),
                                                  Dot_loss_norm(codet54vdcwe_base_reveal_train[i],codet54vdcwe_base_reveal_test[i])]
    
    codet54vdcwe_large_reveal_train[i] = np.load(codet54vdcwe_large+trained_reveal_path+'reveal_train{}'.format(i)+'-data.npy')
    codet54vdcwe_large_reveal_test[i] = np.load(codet54vdcwe_large+trained_reveal_path+'reveal_test{}'.format(i)+'-data.npy')
    codet54vdcwe_large_reveal_distance[i] = [mmd_rbf(codet54vdcwe_large_reveal_train[i],codet54vdcwe_large_reveal_test[i]),
                                                  L2_loss(codet54vdcwe_large_reveal_train[i],codet54vdcwe_large_reveal_test[i]),
                                                  L2_loss_norm(codet54vdcwe_large_reveal_train[i],codet54vdcwe_large_reveal_test[i]),
                                                  L2_squared_loss(codet54vdcwe_large_reveal_train[i],codet54vdcwe_large_reveal_test[i]),
                                                  L2_squared_loss_norm(codet54vdcwe_large_reveal_train[i],codet54vdcwe_large_reveal_test[i]),
                                                  Dot_loss(codet54vdcwe_large_reveal_train[i],codet54vdcwe_large_reveal_test[i]),
                                                  Dot_loss_norm(codet54vdcwe_large_reveal_train[i],codet54vdcwe_large_reveal_test[i])]

    polycoder4vdcwe_small_reveal_train[i] = np.load(polycoder4vdcwe_small+trained_reveal_path+'reveal_train{}'.format(i)+'-data.npy')
    polycoder4vdcwe_small_reveal_test[i] = np.load(polycoder4vdcwe_small+trained_reveal_path+'reveal_test{}'.format(i)+'-data.npy')
    polycoder4vdcwe_small_reveal_distance[i] = [mmd_rbf(polycoder4vdcwe_small_reveal_train[i],polycoder4vdcwe_small_reveal_test[i]),
                                                  L2_loss(polycoder4vdcwe_small_reveal_train[i],polycoder4vdcwe_small_reveal_test[i]),
                                                  L2_loss_norm(polycoder4vdcwe_small_reveal_train[i],polycoder4vdcwe_small_reveal_test[i]),
                                                  L2_squared_loss(polycoder4vdcwe_small_reveal_train[i],polycoder4vdcwe_small_reveal_test[i]),
                                                  L2_squared_loss_norm(polycoder4vdcwe_small_reveal_train[i],polycoder4vdcwe_small_reveal_test[i]),
                                                  Dot_loss(polycoder4vdcwe_small_reveal_train[i],polycoder4vdcwe_small_reveal_test[i]),
                                                  Dot_loss_norm(polycoder4vdcwe_small_reveal_train[i],polycoder4vdcwe_small_reveal_test[i])]
        
    
    polycoder4vdcwe_base_reveal_train[i] = np.load(polycoder4vdcwe_base+trained_reveal_path+'reveal_train{}'.format(i)+'-data.npy')
    polycoder4vdcwe_base_reveal_test[i] = np.load(polycoder4vdcwe_base+trained_reveal_path+'reveal_test{}'.format(i)+'-data.npy')
    polycoder4vdcwe_base_reveal_distance[i] = [mmd_rbf(polycoder4vdcwe_base_reveal_train[i],polycoder4vdcwe_base_reveal_test[i]),
                                                  L2_loss(polycoder4vdcwe_base_reveal_train[i],polycoder4vdcwe_base_reveal_test[i]),
                                                  L2_loss_norm(polycoder4vdcwe_base_reveal_train[i],polycoder4vdcwe_base_reveal_test[i]),
                                                  L2_squared_loss(polycoder4vdcwe_base_reveal_train[i],polycoder4vdcwe_base_reveal_test[i]),
                                                  L2_squared_loss_norm(polycoder4vdcwe_base_reveal_train[i],polycoder4vdcwe_base_reveal_test[i]),
                                                  Dot_loss(polycoder4vdcwe_base_reveal_train[i],polycoder4vdcwe_base_reveal_test[i]),
                                                  Dot_loss_norm(polycoder4vdcwe_base_reveal_train[i],polycoder4vdcwe_base_reveal_test[i])]
        
    
    
    polycoder4vdcwe_large_reveal_train[i] = np.load(polycoder4vdcwe_large+trained_reveal_path+'reveal_train{}'.format(i)+'-data.npy')
    polycoder4vdcwe_large_reveal_test[i] = np.load(polycoder4vdcwe_large+trained_reveal_path+'reveal_test{}'.format(i)+'-data.npy')
    polycoder4vdcwe_large_reveal_distance[i] = [mmd_rbf(polycoder4vdcwe_large_reveal_train[i],polycoder4vdcwe_large_reveal_test[i]),
                                                  L2_loss(polycoder4vdcwe_large_reveal_train[i],polycoder4vdcwe_large_reveal_test[i]),
                                                  L2_loss_norm(polycoder4vdcwe_large_reveal_train[i],polycoder4vdcwe_large_reveal_test[i]),
                                                  L2_squared_loss(polycoder4vdcwe_large_reveal_train[i],polycoder4vdcwe_large_reveal_test[i]),
                                                  L2_squared_loss_norm(polycoder4vdcwe_large_reveal_train[i],polycoder4vdcwe_large_reveal_test[i]),
                                                  Dot_loss(polycoder4vdcwe_large_reveal_train[i],polycoder4vdcwe_large_reveal_test[i]),
                                                  Dot_loss_norm(polycoder4vdcwe_large_reveal_train[i],polycoder4vdcwe_large_reveal_test[i])]
    codebert4vdcwe_small_reveal_train[i] = None
    codebert4vdcwe_small_reveal_test[i] = None
    codebert4vdcwe_base_reveal_train[i] = None
    codebert4vdcwe_base_reveal_test[i] = None
    codegen4vdcwe_small_reveal_train[i] = None
    codegen4vdcwe_small_reveal_test[i] = None
    codegen4vdcwe_base_reveal_train[i] = None
    codegen4vdcwe_base_reveal_test[i] = None
    codet54vdcwe_small_reveal_train[i] = None
    codet54vdcwe_small_reveal_test[i] = None
    codet54vdcwe_base_reveal_train[i] = None
    codet54vdcwe_base_reveal_test[i] = None
    codet54vdcwe_large_reveal_train[i] = None
    codet54vdcwe_large_reveal_test[i] = None
    polycoder4vdcwe_small_reveal_train[i] = None
    polycoder4vdcwe_small_reveal_test[i] = None
    polycoder4vdcwe_base_reveal_train[i] = None
    polycoder4vdcwe_base_reveal_test[i] = None
    polycoder4vdcwe_large_reveal_train[i] = None
    polycoder4vdcwe_large_reveal_test[i] = None          

In [ ]:

codebert4vdcwe_small_cwe119_train = {}
codebert4vdcwe_small_cwe119_test = {}
codebert4vdcwe_base_cwe119_train = {}
codebert4vdcwe_base_cwe119_test = {}
codegen4vdcwe_small_cwe119_train = {}
codegen4vdcwe_small_cwe119_test = {}
codegen4vdcwe_base_cwe119_train = {}
codegen4vdcwe_base_cwe119_test = {}
codet54vdcwe_small_cwe119_train = {}
codet54vdcwe_small_cwe119_test = {}
codet54vdcwe_base_cwe119_train = {}
codet54vdcwe_base_cwe119_test = {}
codet54vdcwe_large_cwe119_train = {}
codet54vdcwe_large_cwe119_test = {}
polycoder4vdcwe_small_cwe119_train = {}
polycoder4vdcwe_small_cwe119_test = {}
polycoder4vdcwe_base_cwe119_train = {}
polycoder4vdcwe_base_cwe119_test = {}
polycoder4vdcwe_large_cwe119_train = {}
polycoder4vdcwe_large_cwe119_test = {}

codebert4vdcwe_small_cwe119_distance = {}
codebert4vdcwe_base_cwe119_distance = {}
codegen4vdcwe_small_cwe119_distance = {}
codegen4vdcwe_base_cwe119_distance = {}
codet54vdcwe_small_cwe119_distance = {}
codet54vdcwe_base_cwe119_distance = {}
codet54vdcwe_large_cwe119_distance = {}
polycoder4vdcwe_small_cwe119_distance = {}
polycoder4vdcwe_base_cwe119_distance = {}
polycoder4vdcwe_large_cwe119_distance = {}

for i in range(200):  
    codebert4vdcwe_small_cwe119_train[i] = np.load(codebert4vdcwe_small+trained_cwe119_path+'cwe119_train{}'.format(i)+'-data.npy')
    codebert4vdcwe_small_cwe119_test[i] = np.load(codebert4vdcwe_small+trained_cwe119_path+'cwe119_test{}'.format(i)+'-data.npy')
    print(np.shape(codebert4vdcwe_small_cwe119_train[i]))  
    print(np.shape(codebert4vdcwe_small_cwe119_test[i]))
    codebert4vdcwe_small_cwe119_distance[i] = [mmd_rbf(codebert4vdcwe_small_cwe119_train[i],codebert4vdcwe_small_cwe119_test[i]),
                                                  L2_loss(codebert4vdcwe_small_cwe119_train[i],codebert4vdcwe_small_cwe119_test[i]),
                                                  L2_loss_norm(codebert4vdcwe_small_cwe119_train[i],codebert4vdcwe_small_cwe119_test[i]),
                                                  L2_squared_loss(codebert4vdcwe_small_cwe119_train[i],codebert4vdcwe_small_cwe119_test[i]),
                                                  L2_squared_loss_norm(codebert4vdcwe_small_cwe119_train[i],codebert4vdcwe_small_cwe119_test[i]),
                                                  Dot_loss(codebert4vdcwe_small_cwe119_train[i],codebert4vdcwe_small_cwe119_test[i]),
                                                  Dot_loss_norm(codebert4vdcwe_small_cwe119_train[i],codebert4vdcwe_small_cwe119_test[i])]
    
    codebert4vdcwe_base_cwe119_train[i] = np.load(codebert4vdcwe_base+trained_cwe119_path+'cwe119_train{}'.format(i)+'-data.npy')
    codebert4vdcwe_base_cwe119_test[i] = np.load(codebert4vdcwe_base+trained_cwe119_path+'cwe119_test{}'.format(i)+'-data.npy')
    print(np.shape(codebert4vdcwe_base_cwe119_train[i]))  
    print(np.shape(codebert4vdcwe_base_cwe119_test[i]))
    codebert4vdcwe_base_cwe119_distance[i] = [mmd_rbf(codebert4vdcwe_base_cwe119_train[i],codebert4vdcwe_base_cwe119_test[i]),
                                                  L2_loss(codebert4vdcwe_base_cwe119_train[i],codebert4vdcwe_base_cwe119_test[i]),
                                                  L2_loss_norm(codebert4vdcwe_base_cwe119_train[i],codebert4vdcwe_base_cwe119_test[i]),
                                                  L2_squared_loss(codebert4vdcwe_base_cwe119_train[i],codebert4vdcwe_base_cwe119_test[i]),
                                                  L2_squared_loss_norm(codebert4vdcwe_base_cwe119_train[i],codebert4vdcwe_base_cwe119_test[i]),
                                                  Dot_loss(codebert4vdcwe_base_cwe119_train[i],codebert4vdcwe_base_cwe119_test[i]),
                                                  Dot_loss_norm(codebert4vdcwe_base_cwe119_train[i],codebert4vdcwe_base_cwe119_test[i])]
              
    codegen4vdcwe_small_cwe119_train[i] = np.load(codegen4vdcwe_small+trained_cwe119_path+'cwe119_train{}'.format(i)+'-data.npy')
    codegen4vdcwe_small_cwe119_test[i] = np.load(codegen4vdcwe_small+trained_cwe119_path+'cwe119_test{}'.format(i)+'-data.npy')
    print(np.shape(codegen4vdcwe_small_cwe119_train[i]))  
    print(np.shape(codegen4vdcwe_small_cwe119_test[i]))
    codegen4vdcwe_small_cwe119_distance[i] = [mmd_rbf(codegen4vdcwe_small_cwe119_train[i],codegen4vdcwe_small_cwe119_test[i]),
                                                  L2_loss(codegen4vdcwe_small_cwe119_train[i],codegen4vdcwe_small_cwe119_test[i]),
                                                  L2_loss_norm(codegen4vdcwe_small_cwe119_train[i],codegen4vdcwe_small_cwe119_test[i]),
                                                  L2_squared_loss(codegen4vdcwe_small_cwe119_train[i],codegen4vdcwe_small_cwe119_test[i]),
                                                  L2_squared_loss_norm(codegen4vdcwe_small_cwe119_train[i],codegen4vdcwe_small_cwe119_test[i]),
                                                  Dot_loss(codegen4vdcwe_small_cwe119_train[i],codegen4vdcwe_small_cwe119_test[i]),
                                                  Dot_loss_norm(codegen4vdcwe_small_cwe119_train[i],codegen4vdcwe_small_cwe119_test[i])]
                            
                           
    
    
    codegen4vdcwe_base_cwe119_train[i] = np.load(codegen4vdcwe_base+trained_cwe119_path+'cwe119_train{}'.format(i)+'-data.npy')
    codegen4vdcwe_base_cwe119_test[i] = np.load(codegen4vdcwe_base+trained_cwe119_path+'cwe119_test{}'.format(i)+'-data.npy')
    print(np.shape(codegen4vdcwe_base_cwe119_train[i]))  
    print(np.shape(codegen4vdcwe_base_cwe119_test[i]))
    codegen4vdcwe_base_cwe119_distance[i] = [mmd_rbf(codegen4vdcwe_base_cwe119_train[i],codegen4vdcwe_base_cwe119_test[i]),
                                                  L2_loss(codegen4vdcwe_base_cwe119_train[i],codegen4vdcwe_base_cwe119_test[i]),
                                                  L2_loss_norm(codegen4vdcwe_base_cwe119_train[i],codegen4vdcwe_base_cwe119_test[i]),
                                                  L2_squared_loss(codegen4vdcwe_base_cwe119_train[i],codegen4vdcwe_base_cwe119_test[i]),
                                                  L2_squared_loss_norm(codegen4vdcwe_base_cwe119_train[i],codegen4vdcwe_base_cwe119_test[i]),
                                                  Dot_loss(codegen4vdcwe_base_cwe119_train[i],codegen4vdcwe_base_cwe119_test[i]),
                                                  Dot_loss_norm(codegen4vdcwe_base_cwe119_train[i],codegen4vdcwe_base_cwe119_test[i])]
        
    codet54vdcwe_small_cwe119_train[i] = np.load(codet54vdcwe_small+trained_cwe119_path+'cwe119_train{}'.format(i)+'-data.npy')
    codet54vdcwe_small_cwe119_test[i] = np.load(codet54vdcwe_small+trained_cwe119_path+'cwe119_test{}'.format(i)+'-data.npy')
    print(np.shape(codet54vdcwe_small_cwe119_train[i]))  
    print(np.shape(codet54vdcwe_small_cwe119_test[i]))
    codet54vdcwe_small_cwe119_distance[i] = [mmd_rbf(codet54vdcwe_small_cwe119_train[i],codet54vdcwe_small_cwe119_test[i]),
                                                  L2_loss(codet54vdcwe_small_cwe119_train[i],codet54vdcwe_small_cwe119_test[i]),
                                                  L2_loss_norm(codet54vdcwe_small_cwe119_train[i],codet54vdcwe_small_cwe119_test[i]),
                                                  L2_squared_loss(codet54vdcwe_small_cwe119_train[i],codet54vdcwe_small_cwe119_test[i]),
                                                  L2_squared_loss_norm(codet54vdcwe_small_cwe119_train[i],codet54vdcwe_small_cwe119_test[i]),
                                                  Dot_loss(codet54vdcwe_small_cwe119_train[i],codet54vdcwe_small_cwe119_test[i]),
                                                  Dot_loss_norm(codet54vdcwe_small_cwe119_train[i],codet54vdcwe_small_cwe119_test[i])]

    codet54vdcwe_base_cwe119_train[i] = np.load(codet54vdcwe_base+trained_cwe119_path+'cwe119_train{}'.format(i)+'-data.npy')
    codet54vdcwe_base_cwe119_test[i] = np.load(codet54vdcwe_base+trained_cwe119_path+'cwe119_test{}'.format(i)+'-data.npy')
    print(np.shape(codet54vdcwe_base_cwe119_train[i]))  
    print(np.shape(codet54vdcwe_base_cwe119_test[i]))
    codet54vdcwe_base_cwe119_distance[i] = [mmd_rbf(codet54vdcwe_base_cwe119_train[i],codet54vdcwe_base_cwe119_test[i]),
                                                  L2_loss(codet54vdcwe_base_cwe119_train[i],codet54vdcwe_base_cwe119_test[i]),
                                                  L2_loss_norm(codet54vdcwe_base_cwe119_train[i],codet54vdcwe_base_cwe119_test[i]),
                                                  L2_squared_loss(codet54vdcwe_base_cwe119_train[i],codet54vdcwe_base_cwe119_test[i]),
                                                  L2_squared_loss_norm(codet54vdcwe_base_cwe119_train[i],codet54vdcwe_base_cwe119_test[i]),
                                                  Dot_loss(codet54vdcwe_base_cwe119_train[i],codet54vdcwe_base_cwe119_test[i]),
                                                  Dot_loss_norm(codet54vdcwe_base_cwe119_train[i],codet54vdcwe_base_cwe119_test[i])]
    
    codet54vdcwe_large_cwe119_train[i] = np.load(codet54vdcwe_large+trained_cwe119_path+'cwe119_train{}'.format(i)+'-data.npy')
    codet54vdcwe_large_cwe119_test[i] = np.load(codet54vdcwe_large+trained_cwe119_path+'cwe119_test{}'.format(i)+'-data.npy')
    print(np.shape(codet54vdcwe_large_cwe119_train[i]))  
    print(np.shape(codet54vdcwe_large_cwe119_test[i]))
    codet54vdcwe_large_cwe119_distance[i] = [mmd_rbf(codet54vdcwe_large_cwe119_train[i],codet54vdcwe_large_cwe119_test[i]),
                                                  L2_loss(codet54vdcwe_large_cwe119_train[i],codet54vdcwe_large_cwe119_test[i]),
                                                  L2_loss_norm(codet54vdcwe_large_cwe119_train[i],codet54vdcwe_large_cwe119_test[i]),
                                                  L2_squared_loss(codet54vdcwe_large_cwe119_train[i],codet54vdcwe_large_cwe119_test[i]),
                                                  L2_squared_loss_norm(codet54vdcwe_large_cwe119_train[i],codet54vdcwe_large_cwe119_test[i]),
                                                  Dot_loss(codet54vdcwe_large_cwe119_train[i],codet54vdcwe_large_cwe119_test[i]),
                                                  Dot_loss_norm(codet54vdcwe_large_cwe119_train[i],codet54vdcwe_large_cwe119_test[i])]

    polycoder4vdcwe_small_cwe119_train[i] = np.load(polycoder4vdcwe_small+trained_cwe119_path+'cwe119_train{}'.format(i)+'-data.npy')
    polycoder4vdcwe_small_cwe119_test[i] = np.load(polycoder4vdcwe_small+trained_cwe119_path+'cwe119_test{}'.format(i)+'-data.npy')
    print(np.shape(polycoder4vdcwe_small_cwe119_train[i]))  
    print(np.shape(polycoder4vdcwe_small_cwe119_test[i]))
    polycoder4vdcwe_small_cwe119_distance[i] = [mmd_rbf(polycoder4vdcwe_small_cwe119_train[i],polycoder4vdcwe_small_cwe119_test[i]),
                                                  L2_loss(polycoder4vdcwe_small_cwe119_train[i],polycoder4vdcwe_small_cwe119_test[i]),
                                                  L2_loss_norm(polycoder4vdcwe_small_cwe119_train[i],polycoder4vdcwe_small_cwe119_test[i]),
                                                  L2_squared_loss(polycoder4vdcwe_small_cwe119_train[i],polycoder4vdcwe_small_cwe119_test[i]),
                                                  L2_squared_loss_norm(polycoder4vdcwe_small_cwe119_train[i],polycoder4vdcwe_small_cwe119_test[i]),
                                                  Dot_loss(polycoder4vdcwe_small_cwe119_train[i],polycoder4vdcwe_small_cwe119_test[i]),
                                                  Dot_loss_norm(polycoder4vdcwe_small_cwe119_train[i],polycoder4vdcwe_small_cwe119_test[i])]
        
    
    polycoder4vdcwe_base_cwe119_train[i] = np.load(polycoder4vdcwe_base+trained_cwe119_path+'cwe119_train{}'.format(i)+'-data.npy')
    polycoder4vdcwe_base_cwe119_test[i] = np.load(polycoder4vdcwe_base+trained_cwe119_path+'cwe119_test{}'.format(i)+'-data.npy')
    print(np.shape(polycoder4vdcwe_base_cwe119_train[i]))  
    print(np.shape(polycoder4vdcwe_base_cwe119_test[i]))
    polycoder4vdcwe_base_cwe119_distance[i] = [mmd_rbf(polycoder4vdcwe_base_cwe119_train[i],polycoder4vdcwe_base_cwe119_test[i]),
                                                  L2_loss(polycoder4vdcwe_base_cwe119_train[i],polycoder4vdcwe_base_cwe119_test[i]),
                                                  L2_loss_norm(polycoder4vdcwe_base_cwe119_train[i],polycoder4vdcwe_base_cwe119_test[i]),
                                                  L2_squared_loss(polycoder4vdcwe_base_cwe119_train[i],polycoder4vdcwe_base_cwe119_test[i]),
                                                  L2_squared_loss_norm(polycoder4vdcwe_base_cwe119_train[i],polycoder4vdcwe_base_cwe119_test[i]),
                                                  Dot_loss(polycoder4vdcwe_base_cwe119_train[i],polycoder4vdcwe_base_cwe119_test[i]),
                                                  Dot_loss_norm(polycoder4vdcwe_base_cwe119_train[i],polycoder4vdcwe_base_cwe119_test[i])]
        
    
    
    polycoder4vdcwe_large_cwe119_train[i] = np.load(polycoder4vdcwe_large+trained_cwe119_path+'cwe119_train{}'.format(i)+'-data.npy')
    polycoder4vdcwe_large_cwe119_test[i] = np.load(polycoder4vdcwe_large+trained_cwe119_path+'cwe119_test{}'.format(i)+'-data.npy')
    print(np.shape(polycoder4vdcwe_large_cwe119_train[i]))  
    print(np.shape(polycoder4vdcwe_large_cwe119_test[i]))
    polycoder4vdcwe_large_cwe119_distance[i] = [mmd_rbf(polycoder4vdcwe_large_cwe119_train[i],polycoder4vdcwe_large_cwe119_test[i]),
                                                  L2_loss(polycoder4vdcwe_large_cwe119_train[i],polycoder4vdcwe_large_cwe119_test[i]),
                                                  L2_loss_norm(polycoder4vdcwe_large_cwe119_train[i],polycoder4vdcwe_large_cwe119_test[i]),
                                                  L2_squared_loss(polycoder4vdcwe_large_cwe119_train[i],polycoder4vdcwe_large_cwe119_test[i]),
                                                  L2_squared_loss_norm(polycoder4vdcwe_large_cwe119_train[i],polycoder4vdcwe_large_cwe119_test[i]),
                                                  Dot_loss(polycoder4vdcwe_large_cwe119_train[i],polycoder4vdcwe_large_cwe119_test[i]),
                                                  Dot_loss_norm(polycoder4vdcwe_large_cwe119_train[i],polycoder4vdcwe_large_cwe119_test[i])]
    codebert4vdcwe_small_cwe119_train[i] = None
    codebert4vdcwe_small_cwe119_test[i] = None
    codebert4vdcwe_base_cwe119_train[i] = None
    codebert4vdcwe_base_cwe119_test[i] = None
    codegen4vdcwe_small_cwe119_train[i] = None
    codegen4vdcwe_small_cwe119_test[i] = None
    codegen4vdcwe_base_cwe119_train[i] = None
    codegen4vdcwe_base_cwe119_test[i] = None
    codet54vdcwe_small_cwe119_train[i] = None
    codet54vdcwe_small_cwe119_test[i] = None
    codet54vdcwe_base_cwe119_train[i] = None
    codet54vdcwe_base_cwe119_test[i] = None
    codet54vdcwe_large_cwe119_train[i] = None
    codet54vdcwe_large_cwe119_test[i] = None
    polycoder4vdcwe_small_cwe119_train[i] = None
    polycoder4vdcwe_small_cwe119_test[i] = None
    polycoder4vdcwe_base_cwe119_train[i] = None
    polycoder4vdcwe_base_cwe119_test[i] = None
    polycoder4vdcwe_large_cwe119_train[i] = None
    polycoder4vdcwe_large_cwe119_test[i] = None       
            

In [ ]:

codebert4vdcwe_small_cwe399_train = {}
codebert4vdcwe_small_cwe399_test = {}
codebert4vdcwe_base_cwe399_train = {}
codebert4vdcwe_base_cwe399_test = {}
codegen4vdcwe_small_cwe399_train = {}
codegen4vdcwe_small_cwe399_test = {}
codegen4vdcwe_base_cwe399_train = {}
codegen4vdcwe_base_cwe399_test = {}
codet54vdcwe_small_cwe399_train = {}
codet54vdcwe_small_cwe399_test = {}
codet54vdcwe_base_cwe399_train = {}
codet54vdcwe_base_cwe399_test = {}
codet54vdcwe_large_cwe399_train = {}
codet54vdcwe_large_cwe399_test = {}
polycoder4vdcwe_small_cwe399_train = {}
polycoder4vdcwe_small_cwe399_test = {}
polycoder4vdcwe_base_cwe399_train = {}
polycoder4vdcwe_base_cwe399_test = {}
polycoder4vdcwe_large_cwe399_train = {}
polycoder4vdcwe_large_cwe399_test = {}

codebert4vdcwe_small_cwe399_distance = {}
codebert4vdcwe_base_cwe399_distance = {}
codegen4vdcwe_small_cwe399_distance = {}
codegen4vdcwe_base_cwe399_distance = {}
codet54vdcwe_small_cwe399_distance = {}
codet54vdcwe_base_cwe399_distance = {}
codet54vdcwe_large_cwe399_distance = {}
polycoder4vdcwe_small_cwe399_distance = {}
polycoder4vdcwe_base_cwe399_distance = {}
polycoder4vdcwe_large_cwe399_distance = {}

for i in range(200):  
    codebert4vdcwe_small_cwe399_train[i] = np.load(codebert4vdcwe_small+trained_cwe399_path+'cwe399_train{}'.format(i)+'-data.npy')
    codebert4vdcwe_small_cwe399_test[i] = np.load(codebert4vdcwe_small+trained_cwe399_path+'cwe399_test{}'.format(i)+'-data.npy')
    print(np.shape(codebert4vdcwe_small_cwe399_train[i]))  
    print(np.shape(codebert4vdcwe_small_cwe399_test[i]))
    codebert4vdcwe_small_cwe399_distance[i] = [mmd_rbf(codebert4vdcwe_small_cwe399_train[i],codebert4vdcwe_small_cwe399_test[i]),
                                                  L2_loss(codebert4vdcwe_small_cwe399_train[i],codebert4vdcwe_small_cwe399_test[i]),
                                                  L2_loss_norm(codebert4vdcwe_small_cwe399_train[i],codebert4vdcwe_small_cwe399_test[i]),
                                                  L2_squared_loss(codebert4vdcwe_small_cwe399_train[i],codebert4vdcwe_small_cwe399_test[i]),
                                                  L2_squared_loss_norm(codebert4vdcwe_small_cwe399_train[i],codebert4vdcwe_small_cwe399_test[i]),
                                                  Dot_loss(codebert4vdcwe_small_cwe399_train[i],codebert4vdcwe_small_cwe399_test[i]),
                                                  Dot_loss_norm(codebert4vdcwe_small_cwe399_train[i],codebert4vdcwe_small_cwe399_test[i])]
    
    codebert4vdcwe_base_cwe399_train[i] = np.load(codebert4vdcwe_base+trained_cwe399_path+'cwe399_train{}'.format(i)+'-data.npy')
    codebert4vdcwe_base_cwe399_test[i] = np.load(codebert4vdcwe_base+trained_cwe399_path+'cwe399_test{}'.format(i)+'-data.npy')
    print(np.shape(codebert4vdcwe_base_cwe399_train[i]))  
    print(np.shape(codebert4vdcwe_base_cwe399_test[i]))
    codebert4vdcwe_base_cwe399_distance[i] = [mmd_rbf(codebert4vdcwe_base_cwe399_train[i],codebert4vdcwe_base_cwe399_test[i]),
                                                  L2_loss(codebert4vdcwe_base_cwe399_train[i],codebert4vdcwe_base_cwe399_test[i]),
                                                  L2_loss_norm(codebert4vdcwe_base_cwe399_train[i],codebert4vdcwe_base_cwe399_test[i]),
                                                  L2_squared_loss(codebert4vdcwe_base_cwe399_train[i],codebert4vdcwe_base_cwe399_test[i]),
                                                  L2_squared_loss_norm(codebert4vdcwe_base_cwe399_train[i],codebert4vdcwe_base_cwe399_test[i]),
                                                  Dot_loss(codebert4vdcwe_base_cwe399_train[i],codebert4vdcwe_base_cwe399_test[i]),
                                                  Dot_loss_norm(codebert4vdcwe_base_cwe399_train[i],codebert4vdcwe_base_cwe399_test[i])]
              
    codegen4vdcwe_small_cwe399_train[i] = np.load(codegen4vdcwe_small+trained_cwe399_path+'cwe399_train{}'.format(i)+'-data.npy')
    codegen4vdcwe_small_cwe399_test[i] = np.load(codegen4vdcwe_small+trained_cwe399_path+'cwe399_test{}'.format(i)+'-data.npy')
    print(np.shape(codegen4vdcwe_small_cwe399_train[i]))  
    print(np.shape(codegen4vdcwe_small_cwe399_test[i]))
    codegen4vdcwe_small_cwe399_distance[i] = [mmd_rbf(codegen4vdcwe_small_cwe399_train[i],codegen4vdcwe_small_cwe399_test[i]),
                                                  L2_loss(codegen4vdcwe_small_cwe399_train[i],codegen4vdcwe_small_cwe399_test[i]),
                                                  L2_loss_norm(codegen4vdcwe_small_cwe399_train[i],codegen4vdcwe_small_cwe399_test[i]),
                                                  L2_squared_loss(codegen4vdcwe_small_cwe399_train[i],codegen4vdcwe_small_cwe399_test[i]),
                                                  L2_squared_loss_norm(codegen4vdcwe_small_cwe399_train[i],codegen4vdcwe_small_cwe399_test[i]),
                                                  Dot_loss(codegen4vdcwe_small_cwe399_train[i],codegen4vdcwe_small_cwe399_test[i]),
                                                  Dot_loss_norm(codegen4vdcwe_small_cwe399_train[i],codegen4vdcwe_small_cwe399_test[i])]
                            
                           
    
    
    codegen4vdcwe_base_cwe399_train[i] = np.load(codegen4vdcwe_base+trained_cwe399_path+'cwe399_train{}'.format(i)+'-data.npy')
    codegen4vdcwe_base_cwe399_test[i] = np.load(codegen4vdcwe_base+trained_cwe399_path+'cwe399_test{}'.format(i)+'-data.npy')
    print(np.shape(codegen4vdcwe_base_cwe399_train[i]))  
    print(np.shape(codegen4vdcwe_base_cwe399_test[i]))
    codegen4vdcwe_base_cwe399_distance[i] = [mmd_rbf(codegen4vdcwe_base_cwe399_train[i],codegen4vdcwe_base_cwe399_test[i]),
                                                  L2_loss(codegen4vdcwe_base_cwe399_train[i],codegen4vdcwe_base_cwe399_test[i]),
                                                  L2_loss_norm(codegen4vdcwe_base_cwe399_train[i],codegen4vdcwe_base_cwe399_test[i]),
                                                  L2_squared_loss(codegen4vdcwe_base_cwe399_train[i],codegen4vdcwe_base_cwe399_test[i]),
                                                  L2_squared_loss_norm(codegen4vdcwe_base_cwe399_train[i],codegen4vdcwe_base_cwe399_test[i]),
                                                  Dot_loss(codegen4vdcwe_base_cwe399_train[i],codegen4vdcwe_base_cwe399_test[i]),
                                                  Dot_loss_norm(codegen4vdcwe_base_cwe399_train[i],codegen4vdcwe_base_cwe399_test[i])]
        
    codet54vdcwe_small_cwe399_train[i] = np.load(codet54vdcwe_small+trained_cwe399_path+'cwe399_train{}'.format(i)+'-data.npy')
    codet54vdcwe_small_cwe399_test[i] = np.load(codet54vdcwe_small+trained_cwe399_path+'cwe399_test{}'.format(i)+'-data.npy')
    print(np.shape(codet54vdcwe_small_cwe399_train[i]))  
    print(np.shape(codet54vdcwe_small_cwe399_test[i]))
    codet54vdcwe_small_cwe399_distance[i] = [mmd_rbf(codet54vdcwe_small_cwe399_train[i],codet54vdcwe_small_cwe399_test[i]),
                                                  L2_loss(codet54vdcwe_small_cwe399_train[i],codet54vdcwe_small_cwe399_test[i]),
                                                  L2_loss_norm(codet54vdcwe_small_cwe399_train[i],codet54vdcwe_small_cwe399_test[i]),
                                                  L2_squared_loss(codet54vdcwe_small_cwe399_train[i],codet54vdcwe_small_cwe399_test[i]),
                                                  L2_squared_loss_norm(codet54vdcwe_small_cwe399_train[i],codet54vdcwe_small_cwe399_test[i]),
                                                  Dot_loss(codet54vdcwe_small_cwe399_train[i],codet54vdcwe_small_cwe399_test[i]),
                                                  Dot_loss_norm(codet54vdcwe_small_cwe399_train[i],codet54vdcwe_small_cwe399_test[i])]

    codet54vdcwe_base_cwe399_train[i] = np.load(codet54vdcwe_base+trained_cwe399_path+'cwe399_train{}'.format(i)+'-data.npy')
    codet54vdcwe_base_cwe399_test[i] = np.load(codet54vdcwe_base+trained_cwe399_path+'cwe399_test{}'.format(i)+'-data.npy')
    print(np.shape(codet54vdcwe_base_cwe399_train[i]))  
    print(np.shape(codet54vdcwe_base_cwe399_test[i]))
    codet54vdcwe_base_cwe399_distance[i] = [mmd_rbf(codet54vdcwe_base_cwe399_train[i],codet54vdcwe_base_cwe399_test[i]),
                                                  L2_loss(codet54vdcwe_base_cwe399_train[i],codet54vdcwe_base_cwe399_test[i]),
                                                  L2_loss_norm(codet54vdcwe_base_cwe399_train[i],codet54vdcwe_base_cwe399_test[i]),
                                                  L2_squared_loss(codet54vdcwe_base_cwe399_train[i],codet54vdcwe_base_cwe399_test[i]),
                                                  L2_squared_loss_norm(codet54vdcwe_base_cwe399_train[i],codet54vdcwe_base_cwe399_test[i]),
                                                  Dot_loss(codet54vdcwe_base_cwe399_train[i],codet54vdcwe_base_cwe399_test[i]),
                                                  Dot_loss_norm(codet54vdcwe_base_cwe399_train[i],codet54vdcwe_base_cwe399_test[i])]
    
    codet54vdcwe_large_cwe399_train[i] = np.load(codet54vdcwe_large+trained_cwe399_path+'cwe399_train{}'.format(i)+'-data.npy')
    codet54vdcwe_large_cwe399_test[i] = np.load(codet54vdcwe_large+trained_cwe399_path+'cwe399_test{}'.format(i)+'-data.npy')
    print(np.shape(codet54vdcwe_large_cwe399_train[i]))  
    print(np.shape(codet54vdcwe_large_cwe399_test[i]))
    codet54vdcwe_large_cwe399_distance[i] = [mmd_rbf(codet54vdcwe_large_cwe399_train[i],codet54vdcwe_large_cwe399_test[i]),
                                                  L2_loss(codet54vdcwe_large_cwe399_train[i],codet54vdcwe_large_cwe399_test[i]),
                                                  L2_loss_norm(codet54vdcwe_large_cwe399_train[i],codet54vdcwe_large_cwe399_test[i]),
                                                  L2_squared_loss(codet54vdcwe_large_cwe399_train[i],codet54vdcwe_large_cwe399_test[i]),
                                                  L2_squared_loss_norm(codet54vdcwe_large_cwe399_train[i],codet54vdcwe_large_cwe399_test[i]),
                                                  Dot_loss(codet54vdcwe_large_cwe399_train[i],codet54vdcwe_large_cwe399_test[i]),
                                                  Dot_loss_norm(codet54vdcwe_large_cwe399_train[i],codet54vdcwe_large_cwe399_test[i])]

    polycoder4vdcwe_small_cwe399_train[i] = np.load(polycoder4vdcwe_small+trained_cwe399_path+'cwe399_train{}'.format(i)+'-data.npy')
    polycoder4vdcwe_small_cwe399_test[i] = np.load(polycoder4vdcwe_small+trained_cwe399_path+'cwe399_test{}'.format(i)+'-data.npy')
    print(np.shape(polycoder4vdcwe_small_cwe399_train[i]))  
    print(np.shape(polycoder4vdcwe_small_cwe399_test[i]))
    polycoder4vdcwe_small_cwe399_distance[i] = [mmd_rbf(polycoder4vdcwe_small_cwe399_train[i],polycoder4vdcwe_small_cwe399_test[i]),
                                                  L2_loss(polycoder4vdcwe_small_cwe399_train[i],polycoder4vdcwe_small_cwe399_test[i]),
                                                  L2_loss_norm(polycoder4vdcwe_small_cwe399_train[i],polycoder4vdcwe_small_cwe399_test[i]),
                                                  L2_squared_loss(polycoder4vdcwe_small_cwe399_train[i],polycoder4vdcwe_small_cwe399_test[i]),
                                                  L2_squared_loss_norm(polycoder4vdcwe_small_cwe399_train[i],polycoder4vdcwe_small_cwe399_test[i]),
                                                  Dot_loss(polycoder4vdcwe_small_cwe399_train[i],polycoder4vdcwe_small_cwe399_test[i]),
                                                  Dot_loss_norm(polycoder4vdcwe_small_cwe399_train[i],polycoder4vdcwe_small_cwe399_test[i])]
        
    
    polycoder4vdcwe_base_cwe399_train[i] = np.load(polycoder4vdcwe_base+trained_cwe399_path+'cwe399_train{}'.format(i)+'-data.npy')
    polycoder4vdcwe_base_cwe399_test[i] = np.load(polycoder4vdcwe_base+trained_cwe399_path+'cwe399_test{}'.format(i)+'-data.npy')
    print(np.shape(polycoder4vdcwe_base_cwe399_train[i]))  
    print(np.shape(polycoder4vdcwe_base_cwe399_test[i]))
    polycoder4vdcwe_base_cwe399_distance[i] = [mmd_rbf(polycoder4vdcwe_base_cwe399_train[i],polycoder4vdcwe_base_cwe399_test[i]),
                                                  L2_loss(polycoder4vdcwe_base_cwe399_train[i],polycoder4vdcwe_base_cwe399_test[i]),
                                                  L2_loss_norm(polycoder4vdcwe_base_cwe399_train[i],polycoder4vdcwe_base_cwe399_test[i]),
                                                  L2_squared_loss(polycoder4vdcwe_base_cwe399_train[i],polycoder4vdcwe_base_cwe399_test[i]),
                                                  L2_squared_loss_norm(polycoder4vdcwe_base_cwe399_train[i],polycoder4vdcwe_base_cwe399_test[i]),
                                                  Dot_loss(polycoder4vdcwe_base_cwe399_train[i],polycoder4vdcwe_base_cwe399_test[i]),
                                                  Dot_loss_norm(polycoder4vdcwe_base_cwe399_train[i],polycoder4vdcwe_base_cwe399_test[i])]
        
    
    
    polycoder4vdcwe_large_cwe399_train[i] = np.load(polycoder4vdcwe_large+trained_cwe399_path+'cwe399_train{}'.format(i)+'-data.npy')
    polycoder4vdcwe_large_cwe399_test[i] = np.load(polycoder4vdcwe_large+trained_cwe399_path+'cwe399_test{}'.format(i)+'-data.npy')
    print(np.shape(polycoder4vdcwe_large_cwe399_train[i]))  
    print(np.shape(polycoder4vdcwe_large_cwe399_test[i]))
    polycoder4vdcwe_large_cwe399_distance[i] = [mmd_rbf(polycoder4vdcwe_large_cwe399_train[i],polycoder4vdcwe_large_cwe399_test[i]),
                                                  L2_loss(polycoder4vdcwe_large_cwe399_train[i],polycoder4vdcwe_large_cwe399_test[i]),
                                                  L2_loss_norm(polycoder4vdcwe_large_cwe399_train[i],polycoder4vdcwe_large_cwe399_test[i]),
                                                  L2_squared_loss(polycoder4vdcwe_large_cwe399_train[i],polycoder4vdcwe_large_cwe399_test[i]),
                                                  L2_squared_loss_norm(polycoder4vdcwe_large_cwe399_train[i],polycoder4vdcwe_large_cwe399_test[i]),
                                                  Dot_loss(polycoder4vdcwe_large_cwe399_train[i],polycoder4vdcwe_large_cwe399_test[i]),
                                                  Dot_loss_norm(polycoder4vdcwe_large_cwe399_train[i],polycoder4vdcwe_large_cwe399_test[i])]
    codebert4vdcwe_small_cwe399_train[i] = None
    codebert4vdcwe_small_cwe399_test[i] = None
    codebert4vdcwe_base_cwe399_train[i] = None
    codebert4vdcwe_base_cwe399_test[i] = None
    codegen4vdcwe_small_cwe399_train[i] = None
    codegen4vdcwe_small_cwe399_test[i] = None
    codegen4vdcwe_base_cwe399_train[i] = None
    codegen4vdcwe_base_cwe399_test[i] = None
    codet54vdcwe_small_cwe399_train[i] = None
    codet54vdcwe_small_cwe399_test[i] = None
    codet54vdcwe_base_cwe399_train[i] = None
    codet54vdcwe_base_cwe399_test[i] = None
    codet54vdcwe_large_cwe399_train[i] = None
    codet54vdcwe_large_cwe399_test[i] = None
    polycoder4vdcwe_small_cwe399_train[i] = None
    polycoder4vdcwe_small_cwe399_test[i] = None
    polycoder4vdcwe_base_cwe399_train[i] = None
    polycoder4vdcwe_base_cwe399_test[i] = None
    polycoder4vdcwe_large_cwe399_train[i] = None
    polycoder4vdcwe_large_cwe399_test[i] = None           

In [ ]:
print(codebert4vdcwe_small_cwe399_distance[0])

In [ ]:

df_codebert4vdcwe_small_codexglue_distance = pd.DataFrame.from_dict(codebert4vdcwe_small_codexglue_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_codebert4vdcwe_base_codexglue_distance = pd.DataFrame.from_dict(codebert4vdcwe_base_codexglue_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_codegen4vdcwe_small_codexglue_distance = pd.DataFrame.from_dict(codegen4vdcwe_small_codexglue_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_codegen4vdcwe_base_codexglue_distance = pd.DataFrame.from_dict(codegen4vdcwe_base_codexglue_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_codet54vdcwe_small_codexglue_distance = pd.DataFrame.from_dict(codet54vdcwe_small_codexglue_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_codet54vdcwe_base_codexglue_distance = pd.DataFrame.from_dict(codet54vdcwe_base_codexglue_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_codet54vdcwe_large_codexglue_distance = pd.DataFrame.from_dict(codet54vdcwe_large_codexglue_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_polycoder4vdcwe_small_codexglue_distance = pd.DataFrame.from_dict(polycoder4vdcwe_small_codexglue_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_polycoder4vdcwe_base_codexglue_distance = pd.DataFrame.from_dict(polycoder4vdcwe_base_codexglue_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_polycoder4vdcwe_large_codexglue_distance = pd.DataFrame.from_dict(polycoder4vdcwe_large_codexglue_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])

df_codebert4vdcwe_small_reveal_distance = pd.DataFrame.from_dict(codebert4vdcwe_small_reveal_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_codebert4vdcwe_base_reveal_distance = pd.DataFrame.from_dict(codebert4vdcwe_base_reveal_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_codegen4vdcwe_small_reveal_distance = pd.DataFrame.from_dict(codegen4vdcwe_small_reveal_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_codegen4vdcwe_base_reveal_distance = pd.DataFrame.from_dict(codegen4vdcwe_base_reveal_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_codet54vdcwe_small_reveal_distance = pd.DataFrame.from_dict(codet54vdcwe_small_reveal_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_codet54vdcwe_base_reveal_distance = pd.DataFrame.from_dict(codet54vdcwe_base_reveal_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_codet54vdcwe_large_reveal_distance = pd.DataFrame.from_dict(codet54vdcwe_large_reveal_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_polycoder4vdcwe_small_reveal_distance = pd.DataFrame.from_dict(polycoder4vdcwe_small_reveal_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_polycoder4vdcwe_base_reveal_distance = pd.DataFrame.from_dict(polycoder4vdcwe_base_reveal_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_polycoder4vdcwe_large_reveal_distance = pd.DataFrame.from_dict(polycoder4vdcwe_large_reveal_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])

df_codebert4vdcwe_small_cwe119_distance = pd.DataFrame.from_dict(codebert4vdcwe_small_cwe119_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_codebert4vdcwe_base_cwe119_distance = pd.DataFrame.from_dict(codebert4vdcwe_base_cwe119_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_codegen4vdcwe_small_cwe119_distance = pd.DataFrame.from_dict(codegen4vdcwe_small_cwe119_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_codegen4vdcwe_base_cwe119_distance = pd.DataFrame.from_dict(codegen4vdcwe_base_cwe119_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_codet54vdcwe_small_cwe119_distance = pd.DataFrame.from_dict(codet54vdcwe_small_cwe119_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_codet54vdcwe_base_cwe119_distance = pd.DataFrame.from_dict(codet54vdcwe_base_cwe119_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_codet54vdcwe_large_cwe119_distance = pd.DataFrame.from_dict(codet54vdcwe_large_cwe119_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_polycoder4vdcwe_small_cwe119_distance = pd.DataFrame.from_dict(polycoder4vdcwe_small_cwe119_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_polycoder4vdcwe_base_cwe119_distance = pd.DataFrame.from_dict(polycoder4vdcwe_base_cwe119_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_polycoder4vdcwe_large_cwe119_distance = pd.DataFrame.from_dict(polycoder4vdcwe_large_cwe119_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])

df_codebert4vdcwe_small_cwe399_distance = pd.DataFrame.from_dict(codebert4vdcwe_small_cwe399_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_codebert4vdcwe_base_cwe399_distance = pd.DataFrame.from_dict(codebert4vdcwe_base_cwe399_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_codegen4vdcwe_small_cwe399_distance = pd.DataFrame.from_dict(codegen4vdcwe_small_cwe399_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_codegen4vdcwe_base_cwe399_distance = pd.DataFrame.from_dict(codegen4vdcwe_base_cwe399_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_codet54vdcwe_small_cwe399_distance = pd.DataFrame.from_dict(codet54vdcwe_small_cwe399_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_codet54vdcwe_base_cwe399_distance = pd.DataFrame.from_dict(codet54vdcwe_base_cwe399_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_codet54vdcwe_large_cwe399_distance = pd.DataFrame.from_dict(codet54vdcwe_large_cwe399_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_polycoder4vdcwe_small_cwe399_distance = pd.DataFrame.from_dict(polycoder4vdcwe_small_cwe399_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_polycoder4vdcwe_base_cwe399_distance = pd.DataFrame.from_dict(polycoder4vdcwe_base_cwe399_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])
df_polycoder4vdcwe_large_cwe399_distance = pd.DataFrame.from_dict(polycoder4vdcwe_large_cwe399_distance, orient='index', columns=['MMD', 'L2_loss', 'L2_loss_norm', 'L2s_loss', 'L2s_loss_norm','Dot_loss','Dot_loss_norm'])


In [ ]:

df_codebert4vdcwe_small_distance = pd.concat([df_codebert4vdcwe_small_codexglue_distance, df_codebert4vdcwe_small_reveal_distance, df_codebert4vdcwe_small_cwe119_distance, df_codebert4vdcwe_small_cwe399_distance], ignore_index=True)
df_codebert4vdcwe_base_distance = pd.concat([df_codebert4vdcwe_base_codexglue_distance, df_codebert4vdcwe_base_reveal_distance, df_codebert4vdcwe_base_cwe119_distance, df_codebert4vdcwe_base_cwe399_distance], ignore_index=True)
df_codegen4vdcwe_small_distance = pd.concat([df_codegen4vdcwe_small_codexglue_distance, df_codegen4vdcwe_small_reveal_distance, df_codegen4vdcwe_small_cwe119_distance, df_codegen4vdcwe_small_cwe399_distance], ignore_index=True)
df_codegen4vdcwe_base_distance = pd.concat([df_codegen4vdcwe_base_codexglue_distance, df_codegen4vdcwe_base_reveal_distance, df_codegen4vdcwe_base_cwe119_distance, df_codegen4vdcwe_base_cwe399_distance], ignore_index=True)
df_codet54vdcwe_small_distance = pd.concat([df_codet54vdcwe_small_codexglue_distance, df_codet54vdcwe_small_reveal_distance, df_codet54vdcwe_small_cwe119_distance, df_codet54vdcwe_small_cwe399_distance], ignore_index=True)
df_codet54vdcwe_base_distance = pd.concat([df_codet54vdcwe_base_codexglue_distance, df_codet54vdcwe_base_reveal_distance, df_codet54vdcwe_base_cwe119_distance, df_codet54vdcwe_base_cwe399_distance], ignore_index=True)
df_codet54vdcwe_large_distance = pd.concat([df_codet54vdcwe_large_codexglue_distance, df_codet54vdcwe_large_reveal_distance, df_codet54vdcwe_large_cwe119_distance, df_codet54vdcwe_large_cwe399_distance], ignore_index=True)
df_polycoder4vdcwe_small_distance = pd.concat([df_polycoder4vdcwe_small_codexglue_distance, df_polycoder4vdcwe_small_reveal_distance, df_polycoder4vdcwe_small_cwe119_distance, df_polycoder4vdcwe_small_cwe399_distance], ignore_index=True)
df_polycoder4vdcwe_base_distance = pd.concat([df_polycoder4vdcwe_base_codexglue_distance, df_polycoder4vdcwe_base_reveal_distance, df_polycoder4vdcwe_base_cwe119_distance, df_polycoder4vdcwe_base_cwe399_distance], ignore_index=True)
df_polycoder4vdcwe_large_distance = pd.concat([df_polycoder4vdcwe_large_codexglue_distance, df_polycoder4vdcwe_large_reveal_distance, df_polycoder4vdcwe_large_cwe119_distance, df_polycoder4vdcwe_large_cwe399_distance], ignore_index=True)




print(df_codebert4vdcwe_small_distance)

In [ ]:


df_codebert4vdcwe_small_distance.to_csv('df_codebert4vdcwe_small_distance.csv', index=False)  
df_codebert4vdcwe_base_distance.to_csv('df_codebert4vdcwe_base_distance.csv', index=False)  
df_codegen4vdcwe_small_distance.to_csv('df_codegen4vdcwe_small_distance.csv', index=False)  
df_codegen4vdcwe_base_distance.to_csv('df_codegen4vdcwe_base_distance.csv', index=False)  
df_codet54vdcwe_small_distance.to_csv('df_codet54vdcwe_small_distance.csv', index=False) 
df_codet54vdcwe_base_distance.to_csv('df_codet54vdcwe_base_distance.csv', index=False) 
df_codet54vdcwe_large_distance.to_csv('df_codet54vdcwe_large_distance.csv', index=False) 
df_polycoder4vdcwe_small_distance.to_csv('df_polycoder4vdcwe_small_distance.csv', index=False) 
df_polycoder4vdcwe_base_distance.to_csv('df_polycoder4vdcwe_base_distance.csv', index=False)  
df_polycoder4vdcwe_large_distance.to_csv('df_polycoder4vdcwe_large_distance.csv', index=False) 



